# Time-evolution convergence figures

Works with JL3 on ppmstar.org

In [1]:
%pylab ipympl
from astropy import units
from nugridpy import utils as ut
from scipy import interpolate
import sys
import pickle  
sys.path.insert(0,'/home/user/user/niagara_scratch_sblouin/PyPPM')
from ppmpy import ppm  
logging.getLogger("matplotlib").setLevel(logging.ERROR)
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [10]:
runs = ['V3','V4','V5']
path = {'V3':'niagara_scratch_fherwig/V3-1000xL-1000xK-1152-TyroneM1e4_0.1',
        'V4':'niagara_scratch_fherwig/V4-100xL-100xK-1152-TyroneM1e4_0.1',
        'V5':'niagara_scratch_fherwig/V5-10xL-10xK-1152-TyroneM1e4_0.1'}
ddir = '/home/user/user/'

In [11]:
rp = {}
rph = {}
for run in runs:
    rprof_dir = ddir+path[run]+'/prfs'
    rp[run] = ppm.RprofSet(rprof_dir)
    rph[run] = rp[run].get_history()

927 rprof files found in '/home/user/user/niagara_scratch_fherwig/V3-1000xL-1000xK-1152-TyroneM1e4_0.1/prfs/.
Dump numbers range from 0 to 926.
862 rprof files found in '/home/user/user/niagara_scratch_fherwig/V4-100xL-100xK-1152-TyroneM1e4_0.1/prfs/.
Dump numbers range from 0 to 861.
1205 rprof files found in '/home/user/user/niagara_scratch_fherwig/V5-10xL-10xK-1152-TyroneM1e4_0.1/prfs/.
Dump numbers range from 0 to 1204.


In [4]:
matplotlib.rcParams.update({'font.size': 12})

In [28]:
close(1);figure(1)
sparse = 2
labels = [r'V3 ($1000 \times$)',r'V4 ($100 \times$)',r'V5 ($10\times$)']
for i,run in enumerate(runs):
    dumps = rph[run].get('NDump')
    t_v = [0]
    Ur_v = [0]
    for dump in dumps[::sparse]:
        Ur = rp[run].get('|U|',fname=dump)
        R = rp[run].get('R',fname=dump)
        cp = rp[run].bound_rad(cycles=dump,r_min=5000,r_max=12000,var='|Ut|',criterion='min_grad')
        Hp = rp[run].compute_Hp(fname=dump)
        Hp_val = interp(cp, R[::-1], Hp[::-1])
        Ur_val = interp(cp-Hp_val, R[::-1], Ur[::-1])
        t_min = rph[run].get('time(mins)')
        t_hrs = t_min[dump]/60
        if t_hrs<t_v[-1]:
            continue
        t_v.append(t_hrs)
        Ur_v.append(1000*Ur_val)
    plot(t_v,Ur_v, ut.linestyle(i)[0],color=ut.linestylecb(i)[2],markevery=.2,label=labels[i])
xlim(0,5000)
legend(frameon=False,ncol=1,loc=1)
ylim(0,62)
ax = plt.gca()
ax.tick_params(axis='both',direction='in',which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
xlabel(r'$t$ (h)')
ylabel(r'$|U|$ (km$\,$s$^{-1}$)')
savefig('convergence.pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …